# Sistema de Recomendação

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
%run functions_recomendacao.py

## Importação dos dados e Treinamento

In [2]:
dados = loadmat("dado3.mat")
Y = dados["Y"]
R = dados["R"]
filmes = open("dado4.txt","r").read().split("\n")[:-1]

In [3]:
sem_nota = [[], []]
for i in range(R.shape[0]):
    if sum(R[i,:]) == 0:
        sem_nota[0].append(i)

for j in range(R.shape[1]):
    if sum(R[:,j]) == 0:
        sem_nota[1].append(j)

if len(sem_nota[0]) == 0:
    print("Todos os filmes receberam pelo menos uma nota.")
if len(sem_nota[1]) == 0:
    print("Todos os usuários deram pelo menos uma nota.")

Todos os filmes receberam pelo menos uma nota.
Todos os usuários deram pelo menos uma nota.


In [4]:
Y_norm, Y_media = normalizacao(Y, R)

In [5]:
n_pars = 100
n_iter = 10000
X, theta, res = treinamento(Y_norm, R, n_pars, n_iter)

In [6]:
res

     fun: 2.553904153176448e-07
     jac: array([-2.54426355e-08, -2.82771542e-07,  1.69145084e-07, ...,
       -1.25731531e-07,  1.05045907e-07, -4.63907142e-07])
 message: 'Optimization terminated successfully.'
    nfev: 13608
     nit: 8943
    njev: 13608
  status: 0
 success: True
       x: array([ 0.1782795 , -0.01503959, -0.49062717, ...,  0.21743605,
       -0.07565959,  0.28781975])

## Previsão das notas

In [9]:
notas_prev = (X @ theta.T) 
for i in range(notas_prev.shape[0]):
    notas_prev[i, :] = notas_prev[i, :] + Y_media[i]
    
print("A maior diferença entre uma nota dada pelo usuário e uma nota atribuída pelo algoritmo é: {:.2f}.".format(np.max(np.abs(Y*R - notas_prev*R))))
print("A função de custo (desvio quadrático médio) ótima encontrada pela otimização é: {:.2f}.".format(res['fun']))

A maior diferença entre uma nota dada pelo usuário e uma nota atribuída pelo algoritmo é: 0.00.
A função de custo (desvio quadrático médio) ótima encontrada pela otimização é: 0.00.


In [10]:
notas = Y + notas_prev * (1 - R)
media_filmes = np.mean(notas, axis=1)

In [13]:
df = pd.DataFrame(filmes, columns=['Filme'])
df['Nota média'] = media_filmes
df.sort_values(by=['Nota média'], ascending=False, inplace=True)
df.reset_index(drop=True,inplace=True)
print("OS 10 filmes de maior nota média são:")
df.head(10)


OS 10 filmes de maior nota média são:


,Filme,Nota média
0,"814 Great Day in Harlem, A (1994)",10.153611
1,1201 Marlene Dietrich: Shadow and Light (1996),9.519190
2,1536 Aiqing wansui (1994),9.498675
3,1189 Prefontaine (1997),9.329807
4,1398 Anna (1996),9.236036
5,1653 Entertaining Angels: The Dorothy Day Stor...,9.168946
6,1293 Star Kid (1997),9.167452
7,"1467 Saint of Fort Washington, The (1993)",9.118527
8,1594 Everest (1998),9.102414
9,1122 They Made Me a Criminal (1939),9.090271
